# Lets save our data as a huggingface dataset, so it's quick to reuse



In [1]:
# import your package
%load_ext autoreload
%autoreload 2

from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, format="<level>{message}</level>", level="INFO")

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np


from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import pickle
import hashlib
from pathlib import Path

import transformers
from datasets import Dataset, DatasetInfo, load_from_disk, load_dataset


from tqdm.auto import tqdm
import os, re, sys, collections, functools, itertools, json

transformers.__version__


'4.30.1'

In [3]:
from src.prompts.format import format_guard_prompt, format_multishot
from src.models.load import load_model
from src.datasets.load import ds2df
from src.datasets.load import rows_item
from src.datasets.batch import batch_hidden_states
from src.datasets.batch import get_unique_config_hash, ds_params2fname
from src.datasets.hs import get_choices_as_tokens, default_class2choices, choice2ids, scores2choice_probs

# Params

In [4]:
# Params
BATCH_SIZE = 10  # None # None means auto # 6 gives 16Gb/25GB. where 10GB is the base model. so 6 is 6/15
USE_MCDROPOUT = True

from src.extraction.config import ExtractConfig

cfg = ExtractConfig(
    model="HuggingFaceH4/starchat-beta",
    datasets = ["imdb", "amazon_polarity", "truthful_qa", "super_glue:boolq"],
)


# Model

Chosing:
- https://old.reddit.com/r/LocalLLaMA/wiki/models
- https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
- https://github.com/deep-diver/LLM-As-Chatbot/blob/main/model_cards.json


A uncensored and large coding ones might be best for lying.

In [5]:
model, tokenizer = load_model(model_repo=dataset_params['model_repo'])

changing pad_token_id from None to 0
changing padding_side from right to left
changing truncation_side from right to left
/home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so'), PosixPath('/home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/ubuntu/mambaforge/envs/dlk2/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

GPTBigCodeConfig {
  "_name_or_path": "HuggingFaceH4/starchat-beta",
  "activation_function": "gelu",
  "architectures": [
    "GPTBigCodeForCausalLM"
  ],
  "attention_softmax_in_fp32": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "inference_runner": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_batch_size": null,
  "max_sequence_length": null,
  "model_type": "gpt_bigcode",
  "multi_query": true,
  "n_embd": 6144,
  "n_head": 48,
  "n_inner": 24576,
  "n_layer": 40,
  "n_positions": 8192,
  "pad_key_length": true,
  "pre_allocate_kv_cache": false,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false
  },
  "resid_pdrop": 0.1,


# Load Dataset

In [6]:
dataset = load_dataset(dataset_params['dataset_name'])
dataset

Found cached dataset amazon_polarity (/home/ubuntu/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 3600000
    })
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 400000
    })
})

## Format prompts

The prompt is the thing we most often have to change and debug. So we do it explicitly here.

We do it as transforms on a huggingface dataset.

In this case we use multishot examples from train, and use the test set to generated the hidden states dataset. We will test generalisation on a whole new dataset.


In [7]:
def add_text_col_to_imbd_ds(r):
    return {"text": f"Review Title: {r['title']}\nReview Content: {r['content']}"}

def random_example(example_prompts):
    """gets a random example from shuffled train"""
    ex = dict(content=("blah blah " * 302))
    while (
        len(tokenizer(ex["content"]).input_ids) > 150
    ):  # 95% are below this 250, 70% below 150
        i = np.random.randint(example_prompts.num_rows)
        ex = example_prompts[i]
    return ex

example_prompts = dataset["train"].select(range(200)).map(add_text_col_to_imbd_ds)
get_random_example = functools.partial(random_example, example_prompts=example_prompts)


Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc/cache-14b8893df38b4af0.arrow


In [8]:
def to_lie(label: int, lie: bool):
    if label == "":
        return ""
    else:
        return label ^ lie

def label_to_choice(label: bool) -> str:
    return ["False", "True"][label]


def format_imdb_multishot(input: str, response: str = "", lie: bool = False, fmt_prompt=dataset_params['prompt_fmt'], n_shots=dataset_params['N_SHOTS']) -> str:
    texts, responses = [], []
    for n in range(n_shots - 1):
        row = get_random_example()
        texts.append(row["text"])
        responses.append(label_to_choice(to_lie(row["label"], lie)))
    texts.append(input)

    if isinstance(response, int):
        response = label_to_choice(to_lie(response, lie))
    responses.append(response)
    return format_multishot(texts, responses, fmt_prompt=fmt_prompt)


In [9]:
lie = True
ds = (
    dataset["test"]
    .select(range(dataset_params["N"]))
    .map(add_text_col_to_imbd_ds)
    .map(lambda ex: {"prompt": format_imdb_multishot(ex["text"], lie=True), "lie": lie})
    .map(
        lambda ex: tokenizer(
            ex["prompt"], padding="max_length", max_length=600, truncation=True, add_special_tokens=True,
            # return_tensors="pt",
            return_attention_mask=True,
        ),
        batched=True,
    )
    .map(
        lambda r: {"prompt_truncated": tokenizer.batch_decode(r["input_ids"])},
        batched=True,
    )
)
ds

Map:   0%|          | 0/9009 [00:00<?, ? examples/s]

Map:   0%|          | 0/9009 [00:00<?, ? examples/s]

Map:   0%|          | 0/9009 [00:00<?, ? examples/s]

Map:   0%|          | 0/9009 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'title', 'content', 'text', 'prompt', 'lie', 'input_ids', 'attention_mask', 'prompt_truncated'],
    num_rows: 9009
})

## Save as Huggingface Dataset

In [10]:
config_hash, info_kwargs = get_unique_config_hash(
    format_imdb_multishot, model, tokenizer, ds, dataset_params['N']
)
dataset_name = ds_params2fname(dataset_params) + config_hash
f = f"../.ds/{dataset_name}"
print(f)

../.ds/model-starchat-beta_ds-amazon-polarity_format-guard-prompt_N9009_2shots_d6813d


In [11]:
gen_kwargs = dict(
    model=model,
    tokenizer=tokenizer,
    data=ds,
    n=dataset_params['N'],
    batch_size=BATCH_SIZE,
)
gen_kwargs

{'model': GPTBigCodeForCausalLM(
   (transformer): GPTBigCodeModel(
     (wte): Embedding(49156, 6144)
     (wpe): Embedding(8192, 6144)
     (drop): Dropout(p=0.1, inplace=False)
     (h): ModuleList(
       (0-39): 40 x GPTBigCodeBlock(
         (ln_1): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
         (attn): GPTBigCodeAttention(
           (c_attn): Linear4bit(in_features=6144, out_features=6400, bias=True)
           (c_proj): Linear4bit(in_features=6144, out_features=6144, bias=True)
           (attn_dropout): Dropout(p=0.1, inplace=False)
           (resid_dropout): Dropout(p=0.1, inplace=False)
         )
         (ln_2): LayerNorm((6144,), eps=1e-05, elementwise_affine=True)
         (mlp): GPTBigCodeMLP(
           (c_fc): Linear4bit(in_features=6144, out_features=24576, bias=True)
           (c_proj): Linear4bit(in_features=24576, out_features=6144, bias=True)
           (act): GELUActivation()
           (dropout): Dropout(p=0.1, inplace=False)
         )
    

In [12]:
ds1 = Dataset.from_generator(
    generator=batch_hidden_states,
    info=DatasetInfo(
        description=f"kwargs={info_kwargs} dataset_params={dataset_params}",
        config_name=f,               
    ),
    gen_kwargs=gen_kwargs,
).with_format("numpy")
ds1

Generating train split: 0 examples [00:00, ? examples/s]

get hidden states:   0%|          | 0/901 [00:00<?, ?it/s]

Dataset generator downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/generator/default-4cd9d3e07734db42/0.0.0. Subsequent calls will reuse this data.


Dataset({
    features: ['hs0', 'scores0', 'hs1', 'scores1', 'label_b', 'ds_index', 'label', 'prompt', 'lie', 'prompt_truncated'],
    num_rows: 9009
})

## Add labels

For our probe. Given next_token scores (logits) we take only the subset the corresponds to our negative tokens (e.g. False, no, ...) and positive tokens (e.g. Yes, yes, affirmative, ...).


In [13]:
class2_ids = choice2ids(tokenizer, dataset_params['choices'])
add_txt_ans0 = lambda r: {'txt_ans0': tokenizer.decode(r['scores0'].argmax(-1))}
add_txt_ans1 = lambda r: {'txt_ans1': tokenizer.decode(r['scores1'].argmax(-1))}
add_ans = lambda r: scores2choice_probs(r, class2_ids)

ds3 = (
    ds1
    .map(add_ans)
    .map(add_txt_ans0)
    .map(add_txt_ans1)
)
ds3

Map:   0%|          | 0/9009 [00:00<?, ? examples/s]

Map:   0%|          | 0/9009 [00:00<?, ? examples/s]

Map:   0%|          | 0/9009 [00:00<?, ? examples/s]

Dataset({
    features: ['hs0', 'scores0', 'hs1', 'scores1', 'label_b', 'ds_index', 'label', 'prompt', 'lie', 'prompt_truncated', 'choice_probs0', 'ans0', 'choice_probs1', 'ans1', 'txt_ans0', 'txt_ans1'],
    num_rows: 9009
})

## Save to disk

In [14]:
ds3.save_to_disk(f)
f

Saving the dataset (0/40 shards):   0%|          | 0/9009 [00:00<?, ? examples/s]

'../.ds/model-starchat-beta_ds-amazon-polarity_format-guard-prompt_N9009_2shots_d6813d'

# QC

In [15]:
ds4 = load_from_disk(f)
ds4

Dataset({
    features: ['hs0', 'scores0', 'hs1', 'scores1', 'label_b', 'ds_index', 'label', 'prompt', 'lie', 'prompt_truncated', 'choice_probs0', 'ans0', 'choice_probs1', 'ans1', 'txt_ans0', 'txt_ans1'],
    num_rows: 9009
})

In [16]:

# QC, check which answers are most common
common_answers = pd.Series(ds4['txt_ans1']).value_counts()
display('Remember it should be binary. Found common LLM answers:', common_answers)

# list unexpected answers
class2choices = dataset_params['choices']
current_choices = set(class2choices[0]+class2choices[1])
unexpected_answers = set(common_answers.head(10).index)-current_choices
if len(unexpected_answers):
    logger.warning(f'found unexpected answers: {unexpected_answers}. You may want to add them to class2choices')
    
mean_prob = ds4['choice_probs1'].sum(-1).mean()
print('mean_prob', mean_prob)
assert ds4['choice_probs1'].sum(-1).mean()>0.4, f"""
Our choices should cover most common answers. But they accounted for a mean probability of {mean_prob:2.2%} (should be >40%). 

To fix this you might want to improve your prompt or add to your choices
"""

'Remember it should be binary. Found common LLM answers:'

False       2645
True        2107
Positive    1638
The         1332
Negative     792
            ... 
S              1
<|end|>        1
Oh             1
Don            1
Pe             1
Name: count, Length: 70, dtype: int64

found unexpected answers: {'I', 'The', 'That', 'This', 'It', 'Ab'}. You may want to add them to class2choices


mean_prob 0.468408


In [17]:
df = ds2df(ds4)
df

,label_b,ds_index,label,prompt,lie,prompt_truncated,choice_probs0,ans0,choice_probs1,ans1,txt_ans0,txt_ans1,dir_true,conf,llm_prob,llm_ans,desired_ans
0,1,0,1,<|system|>You are about to immerse yourself in...,True,<|endoftext|><|endoftext|><|endoftext|><|endof...,"[0.21382241, 0.15512827]",0.420447,"[0.067346975, 0.1840437]",0.732073,False,This,0.311627,0.311627,0.576260,True,False
1,1,1,1,<|system|>You are about to immerse yourself in...,True,<|endoftext|><|endoftext|><|endoftext|><|endof...,"[0.07619762, 0.29984444]",0.797348,"[0.19563195, 0.23088205]",0.541311,True,The,-0.256037,0.256037,0.669330,True,False
2,0,2,0,<|system|>You are about to immerse yourself in...,True,<|endoftext|><|endoftext|><|endoftext|><|endof...,"[0.1768313, 0.09951585]",0.360099,"[0.2351863, 0.19894429]",0.458249,The,True,0.098150,0.098150,0.409174,False,True
3,1,3,1,<|system|>You are about to immerse yourself in...,True,<|endoftext|><|endoftext|><|endoftext|><|endof...,"[0.33293533, 0.2322644]",0.410935,"[0.22930245, 0.12280665]",0.348765,False,The,-0.062170,0.062170,0.379850,False,False
4,1,4,1,<|system|>You are about to immerse yourself in...,True,<|endoftext|><|endoftext|><|endoftext|><|endof...,"[0.4733826, 0.22653806]",0.323658,"[0.19953269, 0.08011225]",0.286468,False,The,-0.037190,0.037190,0.305063,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9004,1,9004,1,<|system|>You are about to immerse yourself in...,True,<|endoftext|><|endoftext|><|endoftext|><|endof...,"[0.08002959, 0.2259903]",0.738458,"[0.063743114, 0.23966143]",0.789881,True,True,0.051423,0.051423,0.764170,True,False
9005,1,9005,1,<|system|>You are about to immerse yourself in...,True,<|endoftext|><|endoftext|><|endoftext|><|endof...,"[0.13560745, 0.4700992]",0.776104,"[0.36183098, 0.36576247]",0.502695,Positive,False,-0.273409,0.273409,0.639399,True,False
9006,1,9006,1,<|system|>You are about to immerse yourself in...,True,<|endoftext|><|endoftext|><|endoftext|><|endof...,"[0.09060982, 0.16063938]",0.639337,"[0.100443915, 0.2047279]",0.670839,True,True,0.031502,0.031502,0.655088,True,False
9007,1,9007,1,<|system|>You are about to immerse yourself in...,True,<|endoftext|><|endoftext|><|endoftext|><|endof...,"[0.23724256, 0.30012703]",0.558501,"[0.37843844, 0.20676398]",0.353314,Positive,False,-0.205187,0.205187,0.455908,False,False


In [18]:
# QC check accuracy
# it should manage to lie some of the time when asked to lie. Many models wont lie unless very explicitly asked to, but we don't want to do that, we want to leave some ambiguity in the prompt

d = df.query('lie==True')
acc = (d.desired_ans==d.llm_ans).mean()
print(f"when the model tries to lie... we get this acc {acc:2.2f}")
assert acc>0.1, f"should be acc>0.1 but is acc={acc}"

when the model tries to lie... we get this acc 0.24


In [19]:
# QC by viewing a row
r = ds4[0]
print(r['prompt_truncated'])
print(r['txt_ans1'])

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|

# QC: generation

Let's a quick generation, so we can QC the output and sanity check that the model can actually do the task

In [20]:
# r = ds[2]
# q = r["prompt_truncated"]

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
# )
# sequences = pipeline(
#     q.lstrip('<|endoftext|>'),
#     max_length=100,
#     do_sample=False,
#     return_full_text=False,
#     eos_token_id=tokenizer.eos_token_id,
# )

# for seq in sequences:
#     print("-" * 80)
#     print(q)
#     print("-" * 80)
#     print(f"`{seq['generated_text']}`")
#     print("-" * 80)
#     print("label", r['label'])


# QC: linear probe

In [21]:
hs = ds4['hs1']-ds4['hs0']
X = hs.reshape(hs.shape[0], -1)
y = (ds4['ans1'] - ds4['ans0'])>0

In [22]:
true_switch_sign = ds4['label']*2-1
# true_switch_sign = ds4['true'][:, 0]*2-1
y = ((ds4['ans1'] - ds4['ans0']) * true_switch_sign) > 0

In [23]:
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

# # Define X and y
# X = dm.hs1-dm.hs2
# y = dm.y>0

# split
n = len(y)
max_rows = 1000
print('split size', n//2)
X_train, X_test = X[:n//2], X[n//2:]
y_train, y_test = y[:n//2], y[n//2:]
X_train = X_train[:max_rows]
y_train = y_train[:max_rows]
X_test = X_test[:max_rows]
y_test = y_test[:max_rows]

# scale
scaler = RobustScaler()
scaler.fit(X_train)
X_train2 = scaler.transform(X_train)
X_test2 = scaler.transform(X_test)
print('lr')

lr = LogisticRegression(class_weight="balanced", penalty="l2", max_iter=380)
lr.fit(X_train2, y_train>0)

split size 4504
lr


LogisticRegression(class_weight='balanced', max_iter=380)

In [24]:
print("Logistic cls acc: {:2.2%} [TRAIN]".format(lr.score(X_train2, y_train>0)))
print("Logistic cls acc: {:2.2%} [TEST]".format(lr.score(X_test2, y_test>0)))

Logistic cls acc: 100.00% [TRAIN]
Logistic cls acc: 57.70% [TEST]
